In [11]:
import csv
from sklearn.model_selection import train_test_split
import pandas as pd

In [4]:
badwords = ['nigger', 'fuck', 'sex', 'negro', 'cunt', 'bitch', 'cum', 'gay', 'dyke', 'fag', 'faggot']
redditwords = ['Edit:', 'edit:']


def check_bad_word(badwordlist, sentence):
    sentencecopy = sentence.split(' ')
    for word in badwords:
        if word in sentencecopy:
            return False
    return True


In [77]:
def filter_csv(csv_file, length_limit=250, is_qa = False):
    if '.csv' in csv_file:
        data = pd.read_csv(csv_file)
    elif '.tsv' in csv_file:
        df = pd.read_csv(csv_file,sep="\t")

    data = data.drop(["Unnamed: 0"], axis=1)
    is_good_list = []
    for i in range(len(data)):
        title = data.iloc[i]['title']
        body = data.iloc[i]['body']
        sentence = title + body
        data.at[i,'title'] = title.replace('\n','')
        data.at[i,'body'] = body.replace('\n','')
        for word in redditwords:
            if word in body:
                data.at[i,'body'] = body[:body.index(word)]
        is_good = check_bad_word(badwords, sentence) and (len(sentence) < length_limit) and (title not in data['title'])
        if is_qa:
            is_good = is_good and sentence.count('?') == 1

        is_good_list.append(is_good)
    data['good'] = is_good_list
    data = data[data['good']]
    data = data.drop(['good'], axis=1)
    return data

In [78]:
filtered = filter_csv("jokes_subreddit.csv")